SETUP THE ENVIRONMENT

In [ ]:
!pip install -U pip
!pip install accelerate
!pip install appdirs
!pip install bitsandbytes
!pip install datasets
!pip install fire
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torch
!pip install sentencepiece
!pip install tensorboardX
!pip install gradio
!pip install -U transformers tokenizers
!pip install scipy
!pip install seaborn


IMPORT AND INITIALISE MODEL AND TOKENIZER

In [ ]:
from datasets import load_from_disk
from huggingface_hub import notebook_login
import gc
notebook_login()


from transformers import AdamW,get_scheduler
import torch
from tqdm.auto import tqdm
import os
import transformers
import textwrap
from transformers import Trainer
from datasets import load_dataset
from transformers import DataCollatorWithPadding
import os
import sys
from typing import List

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)
from torch.utils.data import DataLoader

import fire
import torch
from datasets import load_dataset
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams
import json
import datasets
import torch
from datasets import load_dataset
from transformers import LlamaTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification, \
    DataCollatorForTokenClassification

%matplotlib inline
sns.set(rc={'figure.figsize':(8, 6)})
sns.set(rc={'figure.dpi':100})
sns.set(style='white', palette='muted', font_scale=1.2)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

candidate_labels = ["attack", "normal"]
model = AutoModelForSequenceClassification.from_pretrained("togethercomputer/LLaMA-2-7B-32K",
                                                           load_in_8bit=True,
                                                           torch_dtype=torch.float16,
                                                           device_map="auto", num_labels=2)

tokenizer = LlamaTokenizer.from_pretrained("togethercomputer/LLaMA-2-7B-32K")


LOAD THE DATASET FROM HUGGINGFACE ( ELSE USE THE ONE ALREADY MAPPED AND STORED)

In [ ]:
# Prepare datasets
# #LABELS ALREADY IN PLACE , tokenize the input strings


# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# def tokenize_function(example):
#     return tokenizer(example["text"],truncation=True, padding=False)


# model_features = datasets.Features(
#         {'text': datasets.Value('string'), 'label': datasets.ClassLabel(num_classes=2,names=candidate_labels)})




# train_data2 = load_dataset("niting3c/malicious-packet-analysis",data_dir="network-packet-flow-header-payload",split="train",features=model_features).map(tokenize_function,batched=True)

# val_data = load_dataset("niting3c/malicious-packet-analysis",data_dir="network-packet-flow-header-payload",split="test",features=model_features).map(tokenize_function,batched=True)

# test_data = load_dataset("niting3c/malicious-packet-analysis",data_dir="metasploitable-data",split="test",features=model_features).map(tokenize_function,batched=True)
# train_data1 = load_dataset("niting3c/malicious-packet-analysis",data_dir="normal_netresc",features=model_features).map(tokenize_function,batched=True)
# try:
#     test_data = test_data.remove_columns('text')
#     test_data = test_data.rename_column("label", "labels")
#     test_data.set_format("torch")
#     test_data.column_names
# except ValueError as e :
#     print(e)


# try:
#     train_data2 = train_data2.remove_columns('text')
#     train_data2 = train_data2.rename_column("label", "labels")
#     train_data2.set_format("torch")
#     train_data2.column_names
# except ValueError as e :
#     print(e)
# try:
#     train_data1 = train_data1.remove_columns('text')
#     train_data1 = train_data1.rename_column("label", "labels")
#     train_data1.set_format("torch")
#     train_data1.column_names
# except ValueError as e :
#     print(e)
# try:
#     val_data = val_data.remove_columns('text')
#     val_data = val_data.rename_column("label", "labels")
#     val_data.set_format("torch")
#     val_data.column_names
# except ValueError as e :
#     print(e)


USING THE STORED DATA SET AND LOADING FROM DISK TO SAVE COMPUTE

In [ ]:

train_data1=load_from_disk("data/train_data1")["train"]
train_data2=load_from_disk("data/train_data2")
val_data=load_from_disk("data/val_data")
test_data=load_from_disk("data/test_data")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_data_1_loader = DataLoader(
    train_data1, shuffle=True, batch_size=2, collate_fn=data_collator
)

train_data_2_loader  = DataLoader(
    train_data2, shuffle=True, batch_size=2, collate_fn=data_collator
)

test_data_loader = DataLoader(
    test_data,  batch_size=2, collate_fn=data_collator
)

val_data_loader = DataLoader(
    val_data, batch_size=2, collate_fn=data_collator
)

for batch in train_data_1_loader:
    break
{k: v.shape for k, v in batch.items()}

CLEAR SOME CACHE AND MEMORY

In [ ]:
torch.cuda.empty_cache()
gc.collect()
!nvidia-smi


START TRAINING THE MODEL


In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "heuristic"
model.train()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

device

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_data_1_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)


In [ ]:
for epoch in range(num_epochs):
    for batch in train_data_1_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)